<h2>Find coordinates of neighborhoods' centers and add 'hover' data</h2>

In [62]:
from geojson_utils import point_in_multipolygon
from geojson_utils import centroid
import pandas as pd
import json

In [63]:
data= pd.read_json('data/NeighborhoodTabulationAreas.geojson')

In [64]:
centers = pd.DataFrame(columns=['ntaname', 'center'])
for element in data['features']:
    box_str = str(element['geometry']['coordinates'][0])
    box_str = '{"type": "Polygon","coordinates": '+box_str+'}'
    box = json.loads(box_str)
    point = centroid(box)
    centers = centers.append({'ntaname': element['properties']['ntaname'].lower(), 'center': point['coordinates']}, ignore_index=True)

In [65]:
centers

,ntaname,center
0,borough park,"[-73.98866118244146, 40.63094962893662]"
1,murray hill,"[-73.80954591780244, 40.76835159714577]"
2,east elmhurst,"[-73.86839572917839, 40.763352273402226]"
3,hollis,"[-73.76113710301854, 40.71063936265136]"
4,manhattanville,"[-73.9537819286583, 40.81797553106497]"
...,...,...
190,ozone park,"[-73.84704886355445, 40.67559947721649]"
191,lindenwood-howard beach,"[-73.85896092897983, 40.652852350618424]"
192,university heights-morris heights,"[-73.91597437354349, 40.85252212758218]"
193,new brighton-silver lake,"[-74.10286121893223, 40.630050089062976]"


In [74]:
df = pd.read_csv('data/MergedTreesPropertiesData.csv')

In [75]:
df = pd.merge(df, centers, how='outer', on='ntaname')

In [76]:
df.drop(columns={'Unnamed: 0'}, inplace=True)
df.dropna(inplace=True)


In [77]:
df['hover']=''

In [78]:
for index, row in df.iterrows():
    new_hover=row['ntaname']+'<br>'+row['borough']+'<br>'+'trees/sq.mile: '+str(row['trees/sq.mile'])+'<br>'+'avg. landprice: '+ str(row['avg.landprice_thous$/acre']) + '<br>'+'trees health: '+str(row['health'])+'/15'
    df.at[index,'hover'] = new_hover


In [79]:
df.drop(columns={'borough_y', 'avtot'}, inplace = True)

In [80]:
df.hover[0]

'airport<br>queens<br>trees/sq.mile: 18.94<br>avg. landprice: 1995.03<br>trees health: 14.32/15'

In [81]:
df.to_csv('data\Trees_Properties_With_Centroids.csv')